In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import ensemble, linear_model
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
import plotly.graph_objs as go
import plotly.offline as py


pd.set_option('display.max_columns', None)  
pd.set_option('display.max_rows', 200)  

## Import the data

In [2]:
train = pd.read_csv('../Bain women datahack - QL/train.csv')
test = pd.read_csv('../Bain women datahack - QL/test.csv')

ss = pd.read_csv('../Bain women datahack - QL/sample_submission.csv')

In [3]:
train.head()

,ID,Day_No,Course_ID,Course_Domain,Course_Type,Short_Promotion,Public_Holiday,Long_Promotion,User_Traffic,Competition_Metric,Sales
0,1,1,1,Development,Course,0,1,1,11004,0.007,81
1,2,2,1,Development,Course,0,0,1,13650,0.007,79
2,3,3,1,Development,Course,0,0,1,11655,0.007,75
3,4,4,1,Development,Course,0,0,1,12054,0.007,80
4,5,5,1,Development,Course,0,0,1,6804,0.007,41


In [4]:
train.Course_Domain.unique()

array(['Development', 'Software Marketing', 'Finance & Accounting',
       'Business'], dtype=object)

In [5]:
train.Course_Type.unique()

array(['Course', 'Program', 'Degree'], dtype=object)

The percentage of missing values

In [6]:
print(train.isnull().sum()/train.shape[0])
print()
print(test.isnull().sum()/test.shape[0])

ID                    0.000000
Day_No                0.000000
Course_ID             0.000000
Course_Domain         0.000000
Course_Type           0.000000
Short_Promotion       0.000000
Public_Holiday        0.000000
Long_Promotion        0.000000
User_Traffic          0.000000
Competition_Metric    0.003445
Sales                 0.000000
dtype: float64

ID                    0.000000
Day_No                0.000000
Course_ID             0.000000
Course_Domain         0.000000
Course_Type           0.000000
Short_Promotion       0.000000
Public_Holiday        0.000000
Long_Promotion        0.000000
Competition_Metric    0.003333
dtype: float64


Impute the missing values with the mean value

In [7]:
train['Competition_Metric'].fillna(train['Competition_Metric'].mean(), inplace = True)
test['Competition_Metric'].fillna(test['Competition_Metric'].mean(), inplace = True)

Impute the test **`User_Traffic`** with train data

In [8]:
CourseID_Traffic_DF = pd.DataFrame(train.groupby(['Course_ID'])['User_Traffic'].mean())

In [9]:
test = test.join(CourseID_Traffic_DF,on = 'Course_ID', how = 'left')

In [10]:
test.head()

,ID,Day_No,Course_ID,Course_Domain,Course_Type,Short_Promotion,Public_Holiday,Long_Promotion,Competition_Metric,User_Traffic
0,883,883,1,Development,Course,1,0,1,0.007,11882.071429
1,884,884,1,Development,Course,1,0,1,0.007,11882.071429
2,885,885,1,Development,Course,1,0,1,0.007,11882.071429
3,886,886,1,Development,Course,1,0,1,0.007,11882.071429
4,887,887,1,Development,Course,0,0,1,0.007,11882.071429


### Get the target y and combine training and testing


In [11]:
y_train = np.log1p(train['Sales'])
# train.drop(['Sales'], inplace = True, axis = 1)

test['Sales'] = 0

data = pd.concat([train,test])


In [12]:
data.dtypes

Competition_Metric    float64
Course_Domain          object
Course_ID               int64
Course_Type            object
Day_No                  int64
ID                      int64
Long_Promotion          int64
Public_Holiday          int64
Sales                   int64
Short_Promotion         int64
User_Traffic          float64
dtype: object

In [13]:
data.head(3)

,Competition_Metric,Course_Domain,Course_ID,Course_Type,Day_No,ID,Long_Promotion,Public_Holiday,Sales,Short_Promotion,User_Traffic
0,0.007,Development,1,Course,1,1,1,1,81,0,11004.0
1,0.007,Development,1,Course,2,2,1,0,79,0,13650.0
2,0.007,Development,1,Course,3,3,1,0,75,0,11655.0


In [14]:
test.Day_No.describe()

count    36000.000000
mean       912.500000
std         17.318343
min        883.000000
25%        897.750000
50%        912.500000
75%        927.250000
max        942.000000
Name: Day_No, dtype: float64

The test date ranges from day #883 to #942, for 60 days

# Feature engineering

Change the **`Course_Domain`** and **`Course_Type`** to the overall average sales. This is to change the categories to data.

In [15]:
Course_Domain_sales_mean = pd.DataFrame(data.groupby(['Course_Domain'])['Sales'].mean())
Course_Domain_sales_mean.rename(columns = {'Sales':'Course_Domain_sales_mean'}, inplace = True)

Course_Type_sales_mean = pd.DataFrame(data.groupby(['Course_Type'])['Sales'].mean())
Course_Type_sales_mean.rename(columns = {'Sales':'Course_Type_sales_mean'}, inplace = True)


In [16]:
data = data.join(Course_Domain_sales_mean, on = 'Course_Domain',how = 'left')
data = data.join(Course_Type_sales_mean, on = 'Course_Type',how = 'left')

In [17]:
# remove the original category
data.drop(['Course_Domain','Course_Type'], axis = 1, inplace = True)

In [18]:
# averaged sales with groupby course ID and public holiday
data['sales_meanT7_holiday'] = data.groupby(['Course_ID', 'Public_Holiday'])['Sales'].transform(lambda x: x.shift(60).rolling(7).mean())
data['sales_stdT7_holiday'] = data.groupby(['Course_ID', 'Public_Holiday'])['Sales'].transform(lambda x: x.shift(60).rolling(7).std())

data['sales_meanT30_holiday'] = data.groupby(['Course_ID', 'Public_Holiday'])['Sales'].transform(lambda x: x.shift(60).rolling(30).mean())
data['sales_stdT30_holiday'] = data.groupby(['Course_ID', 'Public_Holiday'])['Sales'].transform(lambda x: x.shift(60).rolling(30).std())

data['sales_meanT60_holiday'] = data.groupby(['Course_ID', 'Public_Holiday'])['Sales'].transform(lambda x: x.shift(60).rolling(60).mean())
data['sales_stdT60_holiday'] = data.groupby(['Course_ID', 'Public_Holiday'])['Sales'].transform(lambda x: x.shift(60).rolling(60).std())


In [19]:
# average sales with rolling data
data['sales_rolling_mean_t7'] = data.groupby(['Course_ID'])['Sales'].transform(lambda x: x.shift(60).rolling(7).mean())
data['sales_rolling_std_t7'] = data.groupby(['Course_ID'])['Sales'].transform(lambda x: x.shift(60).rolling(7).std())
data['sales_rolling_mean_t30'] = data.groupby(['Course_ID'])['Sales'].transform(lambda x: x.shift(60).rolling(30).mean())
data['sales_rolling_std_t30'] = data.groupby(['Course_ID'])['Sales'].transform(lambda x: x.shift(60).rolling(30).std())
data['sales_rolling_skew_t30'] = data.groupby(['Course_ID'])['Sales'].transform(lambda x: x.shift(60).rolling(30).skew())
data['sales_rolling_kurt_t30'] = data.groupby(['Course_ID'])['Sales'].transform(lambda x: x.shift(60).rolling(30).kurt())

data['sales_rolling_mean_t60'] = data.groupby(['Course_ID'])['Sales'].transform(lambda x: x.shift(60).rolling(60).mean())
data['sales_rolling_std_t60'] = data.groupby(['Course_ID'])['Sales'].transform(lambda x: x.shift(60).rolling(60).std())
data['sales_rolling_skew_t60'] = data.groupby(['Course_ID'])['Sales'].transform(lambda x: x.shift(60).rolling(60).skew())
data['sales_rolling_kurt_t60'] = data.groupby(['Course_ID'])['Sales'].transform(lambda x: x.shift(60).rolling(60).kurt())

From the **`Day_No`** we can extract the **`date_in_year`**, **`date_in_month`**, and **`date_in_week`**, assuming there is a pattern with 365, 30, and 7 days cycle

In [20]:
# date info
data['date_in_year'] = data.Day_No%365
data['date_in_month'] = data.Day_No%30
data['date_in_week'] = data.Day_No%7

# remove original day_no info
data.drop(['Day_No'], axis = 1, inplace = True)


In [21]:
data.head(100)

,Competition_Metric,Course_ID,ID,Long_Promotion,Public_Holiday,Sales,Short_Promotion,User_Traffic,Course_Domain_sales_mean,Course_Type_sales_mean,sales_meanT7_holiday,sales_stdT7_holiday,sales_meanT30_holiday,sales_stdT30_holiday,sales_meanT60_holiday,sales_stdT60_holiday,sales_rolling_mean_t7,sales_rolling_std_t7,sales_rolling_mean_t30,sales_rolling_std_t30,sales_rolling_skew_t30,sales_rolling_kurt_t30,sales_rolling_mean_t60,sales_rolling_std_t60,sales_rolling_skew_t60,sales_rolling_kurt_t60,date_in_year,date_in_month,date_in_week
0,0.007,1,1,1,1,81,0,11004.0,112.173042,105.913559,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,1
1,0.007,1,2,1,0,79,0,13650.0,112.173042,105.913559,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,2,2
2,0.007,1,3,1,0,75,0,11655.0,112.173042,105.913559,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,3,3
3,0.007,1,4,1,0,80,0,12054.0,112.173042,105.913559,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,4,4
4,0.007,1,5,1,0,41,0,6804.0,112.173042,105.913559,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,5,5
5,0.007,1,6,1,0,62,0,10395.0,112.173042,105.913559,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,6,6
6,0.007,1,7,1,0,122,1,16023.0,112.173042,105.913559,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,7,0
7,0.007,1,8,1,0,114,1,14385.0,112.173042,105.913559,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8,8,1
8,0.007,1,9,1,0,121,1,16485.0,112.173042,105.913559,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9,9,2
9,0.007,1,10,1,0,100,1,13377.0,112.173042,105.913559,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10,10,3


In [22]:
data.dtypes

Competition_Metric          float64
Course_ID                     int64
ID                            int64
Long_Promotion                int64
Public_Holiday                int64
Sales                         int64
Short_Promotion               int64
User_Traffic                float64
Course_Domain_sales_mean    float64
Course_Type_sales_mean      float64
sales_meanT7_holiday        float64
sales_stdT7_holiday         float64
sales_meanT30_holiday       float64
sales_stdT30_holiday        float64
sales_meanT60_holiday       float64
sales_stdT60_holiday        float64
sales_rolling_mean_t7       float64
sales_rolling_std_t7        float64
sales_rolling_mean_t30      float64
sales_rolling_std_t30       float64
sales_rolling_skew_t30      float64
sales_rolling_kurt_t30      float64
sales_rolling_mean_t60      float64
sales_rolling_std_t60       float64
sales_rolling_skew_t60      float64
sales_rolling_kurt_t60      float64
date_in_year                  int64
date_in_month               

In [23]:
# User_Traffic is dropped because it has too much correlation with Sales. Therefore it is considered as a "y" value
drop_list = ['ID','Course_ID','Sales','User_Traffic']
data.drop(drop_list, axis = 1, inplace = True)

In [24]:
train_data = data[:train.shape[0]]
test_data = data[train.shape[0]:]


# Modeling

## LightGBM with Cross validation

In [25]:
categoricals = ['Short_Promotion','Public_Holiday','Long_Promotion','date_in_year','date_in_month','date_in_week']

## Tune hyperparameters

In [26]:
from sklearn.model_selection import GridSearchCV
import lightgbm as lgb

model_lgb = lgb.LGBMRegressor(boosting_type = 'gbdt', learning_rate = 0.05, num_leaves = 136, n_estimators = 90)

params_test1 = {
    'num_leaves': range(130,140,2)
}

params_test2 = {
    'n_estimators': range(30,100,10)
}

params_test3 = {
    'subsample': np.arange(0.5,1,0.1),
    'subsample_freq': range(2,10,2)
}


In [27]:
# gsearch1 = GridSearchCV(estimator = model_lgb, param_grid = params_test3, 
#                        scoring = 'neg_mean_squared_error', cv = 3)

# gsearch1.fit(train_data, y_train)
# print(gsearch1.best_estimator_)
# print(gsearch1.best_score_)

In [28]:
params = {
    'boosting_type': 'gbdt',
    'objective': 'poisson',
    'metric': 'rmse',
    'subsample_freq': 5,
    'learning_rate': 0.05,
    'num_leaves': 2**8,
    'feature_fraction': 0.9,
    'bagging_fraction':0.9,
    'reg_lambda':2,
    'reg_alpha':1
}

In [29]:
from sklearn.model_selection import KFold

kf = KFold ( n_splits = 5, shuffle = True, random_state = 40)


In [30]:
train.shape

(512087, 11)

In [31]:
import lightgbm as lgb

In [32]:
models_lgb = []
for train_index, test_index in kf.split(train_data): # train_index 就是train的index
    train_features = train_data.loc[train_index]
    train_y = y_train.loc[train_index]
    
    test_features = train_data.loc[test_index]
    test_y = y_train.loc[test_index]
    
    d_training = lgb.Dataset(train_features, label = train_y, 
                             categorical_feature = categoricals)
    d_test = lgb.Dataset(test_features, label = test_y, 
                            categorical_feature = categoricals)
    
    model = lgb.train(params, train_set = d_training, num_boost_round = 20000,
                     valid_sets = [d_training, d_test], verbose_eval = 100, 
                     early_stopping_rounds = 50)
    models_lgb.append(model)
#     del train_features, train_y, test_features, test_y, d_training, d_test
    

Training until validation scores don't improve for 50 rounds
[100]	training's rmse: 0.176402	valid_1's rmse: 0.182036
[200]	training's rmse: 0.151028	valid_1's rmse: 0.159898
[300]	training's rmse: 0.138797	valid_1's rmse: 0.149999
[400]	training's rmse: 0.132978	valid_1's rmse: 0.146069
[500]	training's rmse: 0.129557	valid_1's rmse: 0.144285
[600]	training's rmse: 0.126265	valid_1's rmse: 0.142557
[700]	training's rmse: 0.123702	valid_1's rmse: 0.141337
[800]	training's rmse: 0.121647	valid_1's rmse: 0.140497
[900]	training's rmse: 0.119983	valid_1's rmse: 0.13999
[1000]	training's rmse: 0.118394	valid_1's rmse: 0.139539
[1100]	training's rmse: 0.116787	valid_1's rmse: 0.138957
[1200]	training's rmse: 0.115256	valid_1's rmse: 0.138462
[1300]	training's rmse: 0.114121	valid_1's rmse: 0.138266
[1400]	training's rmse: 0.112722	valid_1's rmse: 0.137827
[1500]	training's rmse: 0.111509	valid_1's rmse: 0.137484
[1600]	training's rmse: 0.110298	valid_1's rmse: 0.137147
[1700]	training's rms

Early stopping, best iteration is:
[3744]	training's rmse: 0.0938058	valid_1's rmse: 0.133032
Training until validation scores don't improve for 50 rounds
[100]	training's rmse: 0.174644	valid_1's rmse: 0.178632
[200]	training's rmse: 0.149538	valid_1's rmse: 0.156776
[300]	training's rmse: 0.136929	valid_1's rmse: 0.146569
[400]	training's rmse: 0.131463	valid_1's rmse: 0.142977
[500]	training's rmse: 0.128246	valid_1's rmse: 0.141355
[600]	training's rmse: 0.12512	valid_1's rmse: 0.13968
[700]	training's rmse: 0.122765	valid_1's rmse: 0.138602
[800]	training's rmse: 0.120725	valid_1's rmse: 0.137746
[900]	training's rmse: 0.119225	valid_1's rmse: 0.137316
[1000]	training's rmse: 0.117645	valid_1's rmse: 0.136828
[1100]	training's rmse: 0.1159	valid_1's rmse: 0.136122
[1200]	training's rmse: 0.114381	valid_1's rmse: 0.13559
[1300]	training's rmse: 0.113103	valid_1's rmse: 0.135231
[1400]	training's rmse: 0.11188	valid_1's rmse: 0.134903
[1500]	training's rmse: 0.110646	valid_1's rmse:

In [33]:
results_lgb = []
for model in models_lgb:
    if  results_lgb == []:
        results_lgb = np.expm1(model.predict(test_data, num_iteration=model.best_iteration)) / len(models_lgb)
    else:
        results_lgb += np.expm1(model.predict(test_data, num_iteration=model.best_iteration)) / len(models_lgb)
    del model

In [34]:
results_lgb

array([141.67279403, 120.0289671 , 108.49097282, ..., 158.38137998,
       163.82001975, 184.59993854])

In [35]:
ss.Sales = results_lgb
ss.to_csv("submission_lgb_Yeqing_Xia.csv", index=False)

## XGBoost

In [36]:
import xgboost as xgb
from xgboost import XGBRegressor

params = {'max_depth': 10, 
          'learning_rate': 0.1, 
          'objective': 'reg:squarederror', 
          'subsample': 0.85, 
          'colsample_bytree': 0.85,
          'n_estimators': 4000,
         'eval_metric': 'rmse'
         }


In [37]:
models_xgb = []
for train_index, test_index in kf.split(train_data): # train_index 就是train的index
    train_features = train_data.loc[train_index]
    train_y = y_train.loc[train_index]
    
    test_features = train_data.loc[test_index]
    test_y = y_train.loc[test_index]
    
    d_training = xgb.DMatrix(train_features, label = train_y)
    d_test = xgb.DMatrix(test_features, label = test_y)
    
    evallist = [(d_training, 'train'), (d_test, 'eval')]  # 这里顺序变了，会导致下面early stopping的metric变化，按照后一个做metric
    
    model = xgb.train(params, d_training, num_boost_round = 10000, evals = evallist, 
                      verbose_eval = 100, early_stopping_rounds = 100)
    
    models_xgb.append(model)


[0]	train-rmse:3.80846	eval-rmse:3.80996
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 100 rounds.
[100]	train-rmse:0.13463	eval-rmse:0.148556
[200]	train-rmse:0.116193	eval-rmse:0.136959
[300]	train-rmse:0.107018	eval-rmse:0.132803
[400]	train-rmse:0.100356	eval-rmse:0.130482
[500]	train-rmse:0.094839	eval-rmse:0.129106
[600]	train-rmse:0.090203	eval-rmse:0.128125
[700]	train-rmse:0.086337	eval-rmse:0.127598
[800]	train-rmse:0.083406	eval-rmse:0.127629
Stopping. Best iteration:
[703]	train-rmse:0.086213	eval-rmse:0.12756

[0]	train-rmse:3.80882	eval-rmse:3.80866
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 100 rounds.
[100]	train-rmse:0.13308	eval-rmse:0.146661
[200]	train-rmse:0.116329	eval-rmse:0.136478
[300]	train-rmse:0.106671	eval-rmse:0.131959
[400]	train-rmse:0.10004	eval-rmse:0.129519
[500]	train-rmse:0.094

In [38]:
results_xgb = []
for model in models_xgb:
    if  results_xgb == []:
        results_xgb = np.expm1(model.predict(xgb.DMatrix(test_data))) / len(models_xgb)
    else:
        results_xgb += np.expm1(model.predict(xgb.DMatrix(test_data))) / len(models_xgb)
    del model

In [39]:
results_xgb

array([115.71158, 113.24753, 105.97563, ..., 143.9178 , 150.9237 ,
       163.62325], dtype=float32)

In [40]:
ss.Sales = results_xgb
ss.to_csv("submission_xgb_Yeqing_Xia.csv", index=False)

## CatBoost

In [41]:
from catboost import Pool, CatBoostRegressor


In [47]:
models_cat = []
for train_index, test_index in kf.split(train_data): # train_index 就是train的index
    train_features = train_data.loc[train_index]
    train_y = y_train.loc[train_index]
    
    test_features = train_data.loc[test_index]
    test_y = y_train.loc[test_index]
    
    
    d_training = Pool(train_features, label = train_y)
    d_test = Pool(test_features, label = test_y)
    
    model = CatBoostRegressor(loss_function="RMSE",
                           eval_metric="RMSE",
#                            task_type="GPU",
                           learning_rate=0.05,
                           iterations=1500,
                           l2_leaf_reg=5,
                           random_seed=42,
                           od_type="Iter",
                           depth=10,
                           early_stopping_rounds=1000,
                           border_count=32,
                           subsample = 0.85,
                          )
    model.fit(d_training,
                    eval_set=d_test,
                    use_best_model=True,
                    verbose=100)
    models_cat.append(model)    

0:	learn: 0.4474157	test: 0.4476696	best: 0.4476696 (0)	total: 64.9ms	remaining: 1m 37s
100:	learn: 0.2011888	test: 0.2020773	best: 0.2020773 (100)	total: 6.13s	remaining: 1m 24s
200:	learn: 0.1798476	test: 0.1815364	best: 0.1815364 (200)	total: 12.1s	remaining: 1m 18s
300:	learn: 0.1691168	test: 0.1715177	best: 0.1715177 (300)	total: 18.1s	remaining: 1m 11s
400:	learn: 0.1624693	test: 0.1654908	best: 0.1654908 (400)	total: 24.2s	remaining: 1m 6s
500:	learn: 0.1574480	test: 0.1610297	best: 0.1610297 (500)	total: 30.7s	remaining: 1m 1s
600:	learn: 0.1537256	test: 0.1578380	best: 0.1578380 (600)	total: 37.9s	remaining: 56.6s
700:	learn: 0.1506755	test: 0.1553689	best: 0.1553689 (700)	total: 44.9s	remaining: 51.2s
800:	learn: 0.1481696	test: 0.1534132	best: 0.1534132 (800)	total: 51.5s	remaining: 45s
900:	learn: 0.1459441	test: 0.1516998	best: 0.1516998 (900)	total: 58.4s	remaining: 38.8s
1000:	learn: 0.1438532	test: 0.1500876	best: 0.1500876 (1000)	total: 1m 5s	remaining: 32.4s
1100:	lea

In [48]:
results_cat = []
for model in models_cat:
    if  results_cat == []:
        results_cat = np.expm1(model.predict(test_data)) / len(models_cat)
    else:
        results_cat += np.expm1(model.predict(test_data)) / len(models_cat)
    del model

In [49]:
results_cat

array([122.25848491, 108.23339846, 103.06185584, ..., 144.05492245,
       141.66066357, 146.02729738])

In [50]:
ss.Sales = results_cat
ss.to_csv("submission_cat_Yeqing_Xia.csv", index=False)

In [51]:
ss.Sales = 0.5*results_cat + 0.2*results_xgb + 0.3*results_cat
ss.to_csv("submission_stacked_Yeqing_Xia.csv", index=False)

In [52]:
ss

,ID,Sales
0,883,120.949105
1,884,109.236224
2,885,103.644612
3,886,105.261839
4,887,56.210263
5,888,84.294407
6,889,76.943572
7,890,74.441534
8,891,69.431447
9,892,70.419817
